In [1]:
from morfeus import read_xyz, XTB
# Import the descriptor classes available.
# (Replace these with the actual class names if they differ.)
from morfeus import (
    ConeAngle,
    BiteAngle,
    BuriedVolume,
    Sterimol) #,
#     DispersionDescriptor,
#     LigandSolidAngle,
#     LocalForceConstant,
#     Pyramidalization,
#     SolventAccessibleSurfaceArea,
#     Sterimol,
#     XTBDescriptors
# )


In [1]:
import os
import pandas as pd

def compute_all_features(xyz_file, metal_center_index=1,
                         cone_index=12,
                         bite_angle_indices=(12, 5, 11),
                         bite_angle_inverted_indices=(5, 12, 11),
                         bite_angle_inverted_ref_atoms=[4, 6],
                         buried_volume_index=12,
                         buried_volume_excluded_atoms=[13, 14, 15, 16, 17]):
    """
    Computes all available molecular features from the given XYZ file.
    
    Parameters:
        xyz_file (str): Path to the XYZ file containing the 3D structure.
        metal_center_index (int): Index (1-based) of the metal center in the structure.
        cone_index (int): Atom index used for computing the ligand cone angle.
        bite_angle_indices (tuple): Tuple of indices (a, b, c) for bite angle computation.
        bite_angle_inverted_indices (tuple): Tuple of indices (a, b, c) for inverted bite angle computation.
        bite_angle_inverted_ref_atoms (list): List of reference atom indices for the inverted bite angle.
        buried_volume_index (int): Atom index used in the buried volume calculation.
        buried_volume_excluded_atoms (list): List of atom indices to exclude for buried volume computation.
        
    Returns:
        dict: A dictionary where keys are feature names and values are the computed descriptors.
    """
    # Load atomic symbols and coordinates from the XYZ file
    elements, coordinates = read_xyz(xyz_file)
    
    features = {}
    
    # Compute the ligand cone angle using cone_index
    cone = ConeAngle(elements, coordinates, cone_index)
    features['cone_angle'] = cone.cone_angle

    # Compute bite angle using bite_angle_indices
    bite = BiteAngle(coordinates, *bite_angle_indices)
    features['bite_angle'] = bite.angle  # adjust attribute name if necessary

    # Compute inverted bite angle using bite_angle_inverted_indices and ref_atoms
    bite_inverted = BiteAngle(coordinates, *bite_angle_inverted_indices, ref_atoms=bite_angle_inverted_ref_atoms)
    features['bite_angle_inverted'] = bite_inverted.angle

    # Compute buried volume using buried_volume_index and excluded_atoms
    buried = BuriedVolume(elements, coordinates, buried_volume_index, excluded_atoms=buried_volume_excluded_atoms)
    features['buried_volume'] = buried.fraction_buried_volume  # adjust if property name differs
  
    return features

def compute_features_for_directory(xyz_dir, 
                                   metal_center_index=1,
                                   cone_index=12,
                                   bite_angle_indices=(12, 5, 11),
                                   bite_angle_inverted_indices=(5, 12, 11),
                                   bite_angle_inverted_ref_atoms=[4, 6],
                                   buried_volume_index=12,
                                   buried_volume_excluded_atoms=[13, 14, 15, 16, 17]):
    """
    Computes features for all XYZ files in the given directory and returns a DataFrame.
    
    Parameters:
        xyz_dir (str): Path to the directory containing XYZ files.
        metal_center_index, cone_index, bite_angle_indices, bite_angle_inverted_indices,
        bite_angle_inverted_ref_atoms, buried_volume_index, buried_volume_excluded_atoms:
            Parameters to be passed to compute_all_features.
    
    Returns:
        DataFrame: A pandas DataFrame with one row per XYZ file and columns for each feature.
    """
    features_list = []
    
    # Iterate through all files in the directory
    for filename in os.listdir(xyz_dir):
        if filename.lower().endswith('.xyz'):
            file_path = os.path.join(xyz_dir, filename)
            # Compute features for the file
            features = compute_all_features(
                file_path, 
                metal_center_index=metal_center_index,
                cone_index=cone_index,
                bite_angle_indices=bite_angle_indices,
                bite_angle_inverted_indices=bite_angle_inverted_indices,
                bite_angle_inverted_ref_atoms=bite_angle_inverted_ref_atoms,
                buried_volume_index=buried_volume_index,
                buried_volume_excluded_atoms=buried_volume_excluded_atoms
            )
            # Optionally include the filename as an identifier
            features['filename'] = filename
            features_list.append(features)
    
    # Convert the list of dictionaries to a DataFrame
    features_df = pd.DataFrame(features_list)
    return features_df

# Example usage:
if __name__ == "__main__":
    # Define the directory containing XYZ files
    xyz_directory = r'C:\Users\edens\Documents\GitHub\lucas_project\new_update_logs\xyz_files'
    
    # Define the indices (if different from the defaults, adjust here)
    metal_center_index = 1
    cone_index = 12
    bite_angle_indices = (12, 5, 11)
    bite_angle_inverted_indices = (5, 12, 11)
    bite_angle_inverted_ref_atoms = [4, 6]
    buried_volume_index = 12
    buried_volume_excluded_atoms = [13, 14, 15, 16, 17]
    
    # Compute the features for all XYZ files in the directory
    df_features = compute_features_for_directory(
        xyz_directory,
        metal_center_index=metal_center_index,
        cone_index=cone_index,
        bite_angle_indices=bite_angle_indices,
        bite_angle_inverted_indices=bite_angle_inverted_indices,
        bite_angle_inverted_ref_atoms=bite_angle_inverted_ref_atoms,
        buried_volume_index=buried_volume_index,
        buried_volume_excluded_atoms=buried_volume_excluded_atoms
    )
    
    # Display the DataFrame
    print(df_features)


NameError: name 'read_xyz' is not defined

In [5]:
os.chdir(r'C:\Users\edens\Documents\GitHub\lucas_project\new_update_logs\xyz_files')
df_features.to_csv('morfeus_features.csv', index=False)

In [6]:
def compute_sterimol(xyz_file, sterimol_atoms):
    # Load atomic symbols and coordinates from the XYZ file
    elements, coordinates = read_xyz(xyz_file)
    
    # Compute the Sterimol parameters using the provided atoms
    sterimol_val = Sterimol(elements, coordinates, sterimol_atoms[0],sterimol_atoms[1])
  
    return sterimol_val

def compute_sterimol_dir(xyz_dir, indices):
    sterimol_dict = {}
    # Loop through each file in the directory along with corresponding indices
    for idx, filename in zip(indices, os.listdir(xyz_dir)):
        if filename.lower().endswith('.xyz'):
            file_path = os.path.join(xyz_dir, filename)
            # Compute the sterimol for this file using the given index/indices
            sterimol_val = compute_sterimol(file_path, idx)
            # Add an entry to the dictionary with both the file name and the computed sterimol
            sterimol_dict[filename] = {
                'file_name': filename,
                'sterimol': sterimol_val
            }
    return sterimol_dict


In [7]:
indices=[[10,4],[10,4],[10,4],[10,4],[10,4],[10,4],[10,4],[10,4],[10,4],[10,4],[10,4],[10,4],[10,4],[10,4],[10,4],[10,4]]
sterimol_dict=compute_sterimol_dir(r'C:\Users\edens\Documents\GitHub\lucas_project\Secondary_Sphere\xyz_renumbered-03-10-25-15-34\benzal',indices)

In [20]:
df=pd.DataFrame()
for key, val in sterimol_dict.items():
    df['name']=key
    df['B1']=sterimol_dict[key]['sterimol'].B_1_value
    df['B5']=sterimol_dict[key]['sterimol'].B_5_value
    df['L']=sterimol_dict[key]['sterimol'].L_value

In [21]:
df

,name,B1,B5,L


In [ ]:
import pandas as pd
os.chdir(r'C:\Users\edens\Documents\GitHub\lucas_project\Secondary_Sphere')
# Data extracted from your screenshot
import pandas as pd

# All data in one list of dictionaries
data = [
    {"File": "3-F_optimized.xyz",     "L": 6.84, "B1": 1.70, "B5": 3.84},
    {"File": "3-Cl_optimized.xyz",    "L": 6.84, "B1": 1.72, "B5": 4.48},
    {"File": "3-Br_optimized.xyz",    "L": 6.84, "B1": 1.74, "B5": 4.71},
    {"File": "3-I_optimized.xyz",     "L": 6.95, "B1": 1.78, "B5": 5.02},
    {"File": "3-COOMe_optimized.xyz", "L": 8.84, "B1": 1.76, "B5": 5.43},
    {"File": "3-CF3_optimized.xyz",   "L": 7.47, "B1": 1.92, "B5": 4.91},
    {"File": "3-NO2_optimized.xyz",   "L": 7.41, "B1": 1.70, "B5": 5.00},
    {"File": "4-F_optimized.xyz",     "L": 7.43, "B1": 1.70, "B5": 3.25},
    {"File": "4-Cl_optimized.xyz",    "L": 8.12, "B1": 1.75, "B5": 3.25},
    {"File": "4-Br_optimized.xyz",    "L": 8.38, "B1": 1.85, "B5": 3.26},
    {"File": "4-I_optimized.xyz",     "L": 8.71, "B1": 1.97, "B5": 3.26},
    {"File": "4-COOMe_optimized.xyz", "L": 9.84, "B1": 1.88, "B5": 3.27},
    {"File": "4-CF3_optimized.xyz",   "L": 8.16, "B1": 2.18, "B5": 3.26},
    {"File": "4-CN_optimized.xyz",    "L": 8.78, "B1": 1.70, "B5": 3.26},
    {"File": "3,5-Cl_optimized.xyz",  "L": 6.84, "B1": 1.75, "B5": 4.46},
    {"File": "3,5-Br_target.xyz",     "L": 6.84, "B1": 1.85, "B5": 4.70}
]

# Create a DataFrame
df = pd.DataFrame(data)

# Print the DataFrame to the console
print(df)

# Save the DataFrame to an Excel file
excel_filename = "Sterimol_values.xlsx"
df.to_excel(excel_filename, index=False)

print(f"Excel file saved as: {excel_filename}")


                     File     L    B1    B5
0       3-F_optimized.xyz  6.84  1.70  3.84
1      3-Cl_optimized.xyz  6.84  1.72  4.48
2      3-Br_optimized.xyz  6.84  1.74  4.71
3       3-I_optimized.xyz  6.95  1.78  5.02
4   3-COOMe_optimized.xyz  8.84  1.76  5.43
5     3-CF3_optimized.xyz  7.47  1.92  4.91
6     3-NO2_optimized.xyz  7.41  1.70  5.00
7       4-F_optimized.xyz  7.43  1.70  3.25
8      4-Cl_optimized.xyz  8.12  1.75  3.25
9      4-Br_optimized.xyz  8.38  1.85  3.26
10      4-I_optimized.xyz  8.71  1.97  3.26
11  4-COOMe_optimized.xyz  9.84  1.88  3.27
12    4-CF3_optimized.xyz  8.16  2.18  3.26
13     4-CN_optimized.xyz  8.78  1.70  3.26
14   3,5-Cl_optimized.xyz  6.84  1.75  4.46
15      3,5-Br_target.xyz  6.84  1.85  4.70
Excel file saved as: Sterimol_values.xlsx


: 